In [0]:
pip install "snowflake-snowpark-python[pandas]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/170.6 kB ? eta -:--:--
     ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/170.6 kB 1.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/170.6 kB 844.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 92.2/170.6 kB 1.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/74.8 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/91.2 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of cryptography to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.7 MB ? eta -:--:--
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/2.7 MB 8.3 MB/s eta 0:00:01
   ━━

In [0]:
%restart_python

In [0]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, trim, upper, to_date, sum as _sum


connection_parameters = {
    "account": "EBDQDYC-NB21316",
    "user": "HARSHA",
    "password": "HarshaRadhakrishnan02",
    "role": "ACCOUNTADMIN",
    "warehouse": "CASE_WH",
    "database": "ITTECHGENIE_DB",
    "schema": "PUBLIC"
}
session = Session.builder.configs(connection_parameters).create()


columns = [
    "OrderID", "OrderDate", "MonthOfSale", "CustomerID", "CustomerName",
    "Country", "Region", "City", "Category", "Subcategory",
    "Quantity", "Discount", "Sales", "Profit"
]

df = session.read.option("skip_header", 1).csv("@retail_stage/Retail_Sales.csv")


df = df.to_df(*columns)

df_clean = (
    df.with_column("OrderDate", to_date(trim(col("OrderDate")), "YYYY-MM-DD"))
      .with_column("Category", upper(col("Category")))
      .with_column("Quantity", col("Quantity").cast("INTEGER"))
      .with_column("Discount", col("Discount").cast("FLOAT"))
      .with_column("Sales", col("Sales").cast("FLOAT"))
      .with_column("Profit", col("Profit").cast("FLOAT"))
)


df_summary = df_clean.group_by("Category").agg(_sum(col("Sales")).alias("Category_Sales"))


df_clean.write.save_as_table("RETAIL_SALES_RAW", mode="overwrite")
df_summary.write.save_as_table("RETAIL_SALES_SUMMARY", mode="overwrite")
